#Mounting Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

cd /content/drive/MyDrive/Pedestrian-Synthesis-GAN

[Errno 2] No such file or directory: '/content/drive/MyDrive/Pedestrian-Synthesis-GAN'
/content


In [17]:
cd drive/MyDrive/computer_vision/02.\ 자율주행\ 제조

[Errno 2] No such file or directory: 'drive/MyDrive/computer_vision/02. 자율주행 제조'
/content/drive/.shortcut-targets-by-id/1OKd3huGNt0Jiva78faTc6dvZGcIZyxx_/[수강생공유용]33개 프로젝트_강의자료 코드/02. 자율주행 제조/코드   데이터/03. pedGan/pytorch


In [13]:
cd '코드   데이터'

/content/drive/.shortcut-targets-by-id/1OKd3huGNt0Jiva78faTc6dvZGcIZyxx_/[수강생공유용]33개 프로젝트_강의자료 코드/02. 자율주행 제조/코드   데이터


In [14]:
cd 03.\ pedGan

/content/drive/.shortcut-targets-by-id/1OKd3huGNt0Jiva78faTc6dvZGcIZyxx_/[수강생공유용]33개 프로젝트_강의자료 코드/02. 자율주행 제조/코드   데이터/03. pedGan


In [15]:
cd pytorch

/content/drive/.shortcut-targets-by-id/1OKd3huGNt0Jiva78faTc6dvZGcIZyxx_/[수강생공유용]33개 프로젝트_강의자료 코드/02. 자율주행 제조/코드   데이터/03. pedGan/pytorch


In [28]:
cp -rf Pedestrian-Synthesis-GAN /content/gan

In [29]:
cd /content/gan

/content/gan





#Installing Libraries

In [3]:

!pip install visdom
!pip install dominate
!pip install --upgrade --no-cache-dir gdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408195 sha256=c3ada1de128dd706ab4f50127e04f0edae1cf64d0ce1039cf045965a281eeb7f
  Stored in directory: /root/.cache/pip/wheels/42/29/49/5bed207bac4578e4d2c0c5fc0226bfd33a7e2953ea56356855
Successfully built visdom
  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.6
    Uninstalling gdown-4.6.6:
      Successfully uninstalled gdown-4.6.6


#Importing Libraries

In [30]:
import time
import os
from util import util
import torch
import numpy as np
import ntpath
import time
from util import html
import os.path
import random
import torchvision.transforms as transforms
import torch
# from data.base_dataset import BaseDataset
from data.image_folder import make_dataset
import json
import torch.utils.data
from data.base_data_loader import BaseDataLoader  #REMEMBER TO REMOVE OPT

import torch.utils.data as data
from PIL import Image

%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

import os
from collections import OrderedDict
from torch.autograd import Variable
import util.util as util
from util.image_pool import ImagePool

from copy import deepcopy
from models import networks


from pdb import set_trace as st


#Defining Variables

In [31]:

dataroot='datasets'
batchSize=1 # 메모리사이즈
loadSize=286
fineSize=256
input_nc=3
output_nc=3
ntest=float("inf")
results_dir='./results/'
aspect_ratio=1.0
how_many=200
ngf=64
ndf=64
which_model_netD='basic'
which_model_netG='unet_256'
n_layers_D=3
gpu_ids=[0]
name='Trained_Model'
dataset_mode='aligned'
model='pix2pix'
which_direction='BtoA'
nThreads=2
checkpoints_dir='./checkpoints'
norm='batch'
serial_batches= False
display_winsize=256
display_id=0
display_port=8097
display_single_pane_ncols=0
identity=0.0
no_dropout=False
max_dataset_size=float("inf")
resize_or_crop='resize_and_crop'
no_flip=False
use_spp=True
display_freq=50
print_freq=100
save_latest_freq=800
save_epoch_freq=200
continue_train=False
phase='train'
which_epoch='2800'
niter=200
niter_decay=800
#Total epochs = nitr+nitr_decay
beta1=0.5
lr=0.0002
no_lsgan=True
lambda_A=100
lambda_B=10.0
pool_size=50
no_html=False
isTrain = True





#defining Functions

In [32]:
def create_model():
    model = 'pix2pix'
    dataset_mode='aligned'
    print(model)

    if model == 'pix2pix':
        assert(dataset_mode == 'aligned')
        model = Pix2PixModel()

    else:
        raise ValueError("Model [%s] not recognized." % model)
    model.initialize()
    print("model [%s] was created" % (model.name()))
    return model

def CreateDataLoader():
    data_loader = CustomDatasetDataLoader()
    print(data_loader.name())
    data_loader.initialize()
    return data_loader



class Visualizer():
    def __init__(self):
        # self.opt = opt
        self.display_id = display_id
        self.use_html = isTrain and not no_html
        self.win_size = display_winsize
        self.name = name
        if self.display_id > 0:
            import visdom
            self.vis = visdom.Visdom(port = display_port)
            self.display_single_pane_ncols = display_single_pane_ncols

        if self.use_html:
            self.web_dir = os.path.join(checkpoints_dir, name, 'web')
            self.img_dir = os.path.join(self.web_dir, 'images')
            print('create web directory %s...' % self.web_dir)
            util.mkdirs([self.web_dir, self.img_dir])
        self.log_name = os.path.join(checkpoints_dir, name, 'loss_log.txt')
        with open(self.log_name, "a") as log_file:
            now = time.strftime("%c")
            log_file.write('================ Training Loss (%s) ================\n' % now)

    # |visuals|: dictionary of images to display or save
    def display_current_results(self, visuals, epoch):
        if self.display_id > 0: # show images in the browser
            if self.display_single_pane_ncols > 0:
                h, w = next(iter(visuals.values())).shape[:2]
                table_css = """<style>
    table {border-collapse: separate; border-spacing:4px; white-space:nowrap; text-align:center}
    table td {width: %dpx; height: %dpx; padding: 4px; outline: 4px solid black}
</style>""" % (w, h)
                ncols = self.display_single_pane_ncols
                title = self.name
                label_html = ''
                label_html_row = ''
                nrows = int(np.ceil(len(visuals.items()) / ncols))
                images = []
                idx = 0
                for label, image_numpy in visuals.items():
                    label_html_row += '<td>%s</td>' % label
                    images.append(image_numpy.transpose([2, 0, 1]))
                    idx += 1
                    if idx % ncols == 0:
                        label_html += '<tr>%s</tr>' % label_html_row
                        label_html_row = ''
                white_image = np.ones_like(image_numpy.transpose([2, 0, 1]))*255
                while idx % ncols != 0:
                    images.append(white_image)
                    label_html_row += '<td></td>'
                    idx += 1
                if label_html_row != '':
                    label_html += '<tr>%s</tr>' % label_html_row
                # pane col = image row
                self.vis.images(images, nrow=ncols, win=self.display_id + 1,
                                padding=2, opts=dict(title=title + ' images'))
                label_html = '<table>%s</table>' % label_html
                self.vis.text(table_css + label_html, win = self.display_id + 2,
                              opts=dict(title=title + ' labels'))
            else:
                idx = 1
                for label, image_numpy in visuals.items():
                    #image_numpy = np.flipud(image_numpy)
                    self.vis.image(image_numpy.transpose([2,0,1]), opts=dict(title=label),
                                       win=self.display_id + idx)
                    idx += 1

        if self.use_html: # save images to a html file
            for label, image_numpy in visuals.items():
                img_path = os.path.join(self.img_dir, 'epoch%.3d_%s.png' % (epoch, label))
                util.save_image(image_numpy, img_path)
            # update website
            webpage = html.HTML(self.web_dir, 'Experiment name = %s' % self.name, reflesh=1)
            for n in range(epoch, 0, -1):
                webpage.add_header('epoch [%d]' % n)
                ims = []
                txts = []
                links = []

                for label, image_numpy in visuals.items():
                    img_path = 'epoch%.3d_%s.png' % (n, label)
                    ims.append(img_path)
                    txts.append(label)
                    links.append(img_path)
                webpage.add_images(ims, txts, links, width=self.win_size)
            webpage.save()

    # errors: dictionary of error labels and values
    def plot_current_errors(self, epoch, counter_ratio, opt, errors):
        if not hasattr(self, 'plot_data'):
            self.plot_data = {'X':[],'Y':[], 'legend':list(errors.keys())}
        self.plot_data['X'].append(epoch + counter_ratio)
        self.plot_data['Y'].append([errors[k] for k in self.plot_data['legend']])
        self.vis.line(
            X=np.stack([np.array(self.plot_data['X'])]*len(self.plot_data['legend']),1),
            Y=np.array(self.plot_data['Y']),
            opts={
                'title': self.name + ' loss over time',
                'legend': self.plot_data['legend'],
                'xlabel': 'epoch',
                'ylabel': 'loss'},
            win=self.display_id)

    # errors: same format as |errors| of plotCurrentErrors
    def print_current_errors(self, epoch, i, errors, t):
        message = '(epoch: %d, iters: %d, time: %.3f) ' % (epoch, i, t)
        for k, v in errors.items():
            message += '%s: %.3f ' % (k, v)

        print(message)
        with open(self.log_name, "a") as log_file:
            log_file.write('%s\n' % message)

    # save image to the disk

    def save_images(self, webpage, visuals, image_path):
        id_d=0
        image_dir = webpage.get_image_dir()
        short_path = ntpath.basename(image_path[0])
        name = os.path.splitext(short_path)[0]

        webpage.add_header(name)
        ims = []
        txts = []
        links = []
        for label, image_numpy in visuals.items():
            image_name = '%s_%s.png' % (name, label)
            save_path = os.path.join(image_dir, image_name)
            util.save_image(image_numpy, save_path)
            if id_d==3:
              figure(figsize=(8, 6), dpi=80)
              plt.imshow(image_numpy, interpolation='nearest')
              plt.title('Generated Pedestrian')
              plt.show()

            ims.append(image_name)
            txts.append(label)
            links.append(image_name)
            id_d+=1
        webpage.add_images(ims, txts, links, width=self.win_size)

#Defining DataLoader

In [33]:


class BaseDataset(data.Dataset):
    def __init__(self):
        super(BaseDataset, self).__init__()

    def name(self):
        return 'BaseDataset'

    def initialize(self):
        pass

def get_transform():
    transform_list = []
    if resize_or_crop == 'resize_and_crop':
        osize = [loadSize, loadSize]
        transform_list.append(transforms.Scale(osize, Image.BICUBIC))
        transform_list.append(transforms.RandomCrop(fineSize))
    elif resize_or_crop == 'crop':
        transform_list.append(transforms.RandomCrop(fineSize))
    elif resize_or_crop == 'scale_width':
        transform_list.append(transforms.Lambda(
            lambda img: __scale_width(img, fineSize)))
    elif resize_or_crop == 'scale_width_and_crop':
        transform_list.append(transforms.Lambda(
            lambda img: __scale_width(img, loadSize)))
        transform_list.append(transforms.RandomCrop(fineSize))

    if isTrain and not no_flip:
        transform_list.append(transforms.RandomHorizontalFlip())

    transform_list += [transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5),
                                            (0.5, 0.5, 0.5))]
    return transforms.Compose(transform_list)

def __scale_width(img, target_width):
    ow, oh = img.size
    if (ow == target_width):
        return img
    w = target_width
    h = int(target_width * oh / ow)
    return img.resize((w, h), Image.BICUBIC)




class AlignedDataset(BaseDataset): #noise와 원본데이터필요해야해서?
    def initialize(self): #노이즈는 변하지 않는다.
        # self.opt = opt
        self.root = dataroot
        self.fineSize=fineSize
        self.loadSize=loadSize
        self.no_flip=no_flip
        self.dir_AB = os.path.join(dataroot, 'images', phase)
        self.dir_bbox = os.path.join(dataroot, 'bbox', phase)

        #self.AB_paths, self.bbox_paths = sorted(make_dataset(self.dir_AB, self.dir_bbox))
        self.AB_paths, self.bbox_paths = make_dataset(self.dir_AB, self.dir_bbox)
        self.AB_paths = sorted(self.AB_paths)
        self.bbox_paths = sorted(self.bbox_paths)

        assert(resize_or_crop == 'resize_and_crop')

        transform_list = [transforms.ToTensor(),
                          transforms.Normalize((0.5, 0.5, 0.5),
                                               (0.5, 0.5, 0.5))]

        self.transform = transforms.Compose(transform_list)

    def __getitem__(self, index):
        AB_path = self.AB_paths[index]
        #print(AB_path)
        bbox_path = self.bbox_paths[index]
        #print(bbox_path)

        w_total = self.loadSize * 2
        w = int(w_total / 2)
        h = self.loadSize
        w_offset = random.randint(0, max(0, w - self.fineSize - 1))
        h_offset = random.randint(0, max(0, h - self.fineSize - 1))

        bbox = json.load(open(bbox_path))
        #bbox = [bbox['y'], bbox['x'], bbox['w'], bbox['h']]
        #print(bbox['y'], bbox['x'], bbox['w'], bbox['h'])
        bbox_x = max(int((bbox['x']/self.fineSize)*self.loadSize), 0)
        bbox_y = max(int((bbox['y']/self.fineSize)*self.loadSize), 0)
        bbox_w = max(int((bbox['w']/self.fineSize)*self.loadSize), 0)
        bbox_h = max(int((bbox['h']/self.fineSize)*self.loadSize), 0)

        if bbox_y <= h_offset or bbox_x <= w_offset:
            AB = Image.open(AB_path).convert('RGB')
            AB = AB.resize((self.fineSize * 2, self.fineSize), Image.BICUBIC)
            AB = self.transform(AB)
            A = AB[:, :self.fineSize,
               :self.fineSize]
            B = AB[:, :self.fineSize,
                self.fineSize:2*self.fineSize]
            bbox = [bbox['y'], bbox['x'], bbox['w'], bbox['h']]
        else:
            AB = Image.open(AB_path).convert('RGB')
            AB = AB.resize((self.loadSize * 2, self.loadSize), Image.BICUBIC)
            AB = self.transform(AB)
            # 아래는 random crop
            A = AB[:, h_offset:h_offset + self.fineSize,
               w_offset:w_offset + self.fineSize]
            B = AB[:, h_offset:h_offset + self.fineSize,
                w + w_offset:w + w_offset + self.fineSize]
            bbox = [bbox_y-h_offset, bbox_x-w_offset, bbox_w, bbox_h]
        # print('haha')
        # print(bbox)


        if (not self.no_flip) and random.random() < 0.5:
            idx = [i for i in range(A.size(2) - 1, -1, -1)]
            idx = torch.LongTensor(idx)
            A = A.index_select(2, idx)
            B = B.index_select(2, idx)
            #print A.size(2)
            bbox = [bbox[0], A.size(2) - bbox[2], A.size(2) - bbox[1], bbox[3]]
        # print('hehe')
        # print(bbox)
        #print(A.size())
        return {'A': A, 'B': B, 'bbox': bbox,
                'A_paths': AB_path, 'B_paths': AB_path}

    def __len__(self):
        return len(self.AB_paths)

    def name(self):
        return 'AlignedDataset'




def CreateDataset():
    dataset = None
    if dataset_mode == 'aligned':

        dataset = AlignedDataset()
    elif dataset_mode == 'unaligned':
        from data.unaligned_dataset import UnalignedDataset
        dataset = UnalignedDataset()
    elif dataset_mode == 'single':
        from data.single_dataset import SingleDataset
        dataset = SingleDataset()
    else:
        raise ValueError("Dataset [%s] not recognized." % dataset_mode)

    print("dataset [%s] was created" % (dataset.name()))
    dataset.initialize()
    return dataset


class CustomDatasetDataLoader(BaseDataLoader):
    def name(self):
        return 'CustomDatasetDataLoader'

    def initialize(self):
        BaseDataLoader.initialize(self)
        self.dataset = CreateDataset()
        self.max_dataset_size=max_dataset_size
        self.dataloader = torch.utils.data.DataLoader(
            self.dataset,
            batch_size=batchSize,
            shuffle=not serial_batches,
            num_workers=int(nThreads))

    def load_data(self):
        return self.dataloader

    def __len__(self):
        return min(len(self.dataset), self.max_dataset_size)




#Defining  Model

In [34]:



class BaseModel():
    def name(self):
        return 'BaseModel'

    def initialize(self):
        # self.opt = opt
        self.gpu_ids = gpu_ids
        self.isTrain = isTrain
        self.Tensor = torch.cuda.FloatTensor if self.gpu_ids else torch.Tensor
        self.save_dir = os.path.join(checkpoints_dir, name)

    def set_input(self, input):
        self.input = input

    def forward(self):
        pass

    # used in test time, no backprop
    def test(self):
        pass

    def get_image_paths(self):
        pass

    def optimize_parameters(self):
        pass

    def get_current_visuals(self):
        return self.input

    def get_current_errors(self):
        return {}

    def save(self, label):
        pass

    # helper saving function that can be used by subclasses
    def save_network(self, network, network_label, epoch_label, gpu_ids):
        save_filename = '%s_net_%s.pth' % (epoch_label, network_label)
        save_path = os.path.join(self.save_dir, save_filename)
        torch.save(network.cpu().state_dict(), save_path)
        if len(gpu_ids) and torch.cuda.is_available():
            network.cuda(device=gpu_ids[0])

    # helper loading function that can be used by subclasses
    def load_network(self, network, network_label, epoch_label):
        save_filename = '%s_net_%s.pth' % (epoch_label, network_label)
        save_path = os.path.join(self.save_dir, save_filename)
        network.load_state_dict(torch.load(save_path))

    def update_learning_rate():
        pass





class Pix2PixModel(BaseModel):
    def name(self):
        return 'Pix2PixModel'

    def initialize(self):
        BaseModel.initialize(self)
        # self.opt = opt
        self.lr=lr
        self.niter_decay=niter_decay
        self.isTrain = isTrain
        self.which_direction=which_direction
        # define tensors
        self.lambda_A=lambda_A
        self.input_A = self.Tensor(batchSize, input_nc,
                                   fineSize, fineSize)
        self.input_B = self.Tensor(batchSize, output_nc,
                                   fineSize, fineSize)

        transform_list = [transforms.ToTensor(),
                          transforms.Normalize((0.5, 0.5, 0.5),
                                               (0.5, 0.5, 0.5))]

        self.transform = transforms.Compose(transform_list)

        # load/define networks
        self.netG = networks.define_G(input_nc, output_nc, ngf,
                                      which_model_netG, norm, not no_dropout, self.gpu_ids)
        if self.isTrain:
            use_sigmoid = no_lsgan
            self.netD_image = networks.define_image_D(input_nc + output_nc, ndf,
                                          which_model_netD,
                                          n_layers_D, norm, use_sigmoid, self.gpu_ids)
            use_sigmoid = not no_lsgan
            self.netD_person = networks.define_person_D(input_nc, ndf, use_sigmoid, self.gpu_ids)

        if not self.isTrain or continue_train:
            self.load_network(self.netG, 'G', which_epoch)
            if self.isTrain:
                self.load_network(self.netD_image, 'D_image', which_epoch)
                self.load_network(self.netD_person, 'D_person', which_epoch)

        if self.isTrain:
            self.fake_AB_pool = ImagePool(pool_size)
            self.old_lr = lr
            # define loss functions
            #print('haha'+ str(opt.no_lsgan))
            # self.criterionGAN = networks.GANLoss(use_lsgan=not opt.no_lsgan, tensor=self.Tensor)
            self.criterionGAN_image = networks.GANLoss(use_lsgan=not no_lsgan, tensor=self.Tensor)
            self.criterionGAN_person = networks.GANLoss(use_lsgan=no_lsgan, tensor=self.Tensor)
            self.criterionL1 = torch.nn.L1Loss()

            # initialize optimizers
            self.optimizer_G = torch.optim.Adam(self.netG.parameters(),
                                                lr=lr, betas=(beta1, 0.999))
            self.optimizer_D_image = torch.optim.Adam(self.netD_image.parameters(),
                                                lr=lr, betas=(beta1, 0.999))
            self.optimizer_D_person = torch.optim.Adam(self.netD_person.parameters(),
                                                lr=lr, betas=(beta1, 0.999))

        print('---------- Networks initialized -------------')
        networks.print_network(self.netG)
        if self.isTrain:
            networks.print_network(self.netD_image)
            networks.print_network(self.netD_person)
        print('-----------------------------------------------')

    def set_input(self, input):
        AtoB = self.which_direction == 'AtoB'
        input_A = input['A' if AtoB else 'B']
        input_B = input['B' if AtoB else 'A']
        #print(input_A.size())
        self.bbox = input['bbox']
        self.input_A.resize_(input_A.size()).copy_(input_A)
        self.input_B.resize_(input_B.size()).copy_(input_B)

        self.image_paths = input['A_paths' if AtoB else 'B_paths']

    def forward(self):
        self.real_A = Variable(self.input_A)
        self.fake_B = self.netG.forward(self.real_A)
        self.real_B = Variable(self.input_B)

        y,x,w,h = self.bbox
        self.person_crop_real = self.real_B[:,:,y[0]:h[0],x[0]:w[0]]
        self.person_crop_fake = self.fake_B[:,:,y[0]:h[0],x[0]:w[0]]

    # no backprop gradients
    def test(self):
        self.real_A = Variable(self.input_A, volatile=True)
        self.fake_B = self.netG.forward(self.real_A)
        self.real_B = Variable(self.input_B, volatile=True)

        y,x,w,h = self.bbox
        self.person_crop_real = self.real_B[:,:,y[0]:h[0],x[0]:w[0]]
        self.person_crop_fake = self.fake_B[:,:,y[0]:h[0],x[0]:w[0]]

    # get image paths
    def get_image_paths(self):
        return self.image_paths

    def backward_D_image(self):
        # Fake
        # stop backprop to the generator by detaching fake_B
        fake_AB = self.fake_AB_pool.query(torch.cat((self.real_A, self.fake_B), 1))
        self.pred_fake = self.netD_image.forward(fake_AB.detach())
        # self.loss_D_image_fake = self.criterionGAN(self.pred_fake, False)
        self.loss_D_image_fake = self.criterionGAN_image(self.pred_fake, False)

        # Real
        real_AB = torch.cat((self.real_A, self.real_B), 1)
        self.pred_real = self.netD_image.forward(real_AB)
        # self.loss_D_image_real = self.criterionGAN(self.pred_real, True)
        self.loss_D_image_real = self.criterionGAN_image(self.pred_real, True)

        # Combined loss
        self.loss_D_image = (self.loss_D_image_fake + self.loss_D_image_real) * 0.5

        self.loss_D_image.backward()

    def backward_D_person(self):
        #Fake
        self.person_fake = self.netD_person.forward(self.person_crop_fake)
        # self.loss_D_person_fake = self.criterionGAN(self.person_fake, False)
        self.loss_D_person_fake = self.criterionGAN_person(self.person_fake, False)

        #Real
        self.person_real = self.netD_person.forward(self.person_crop_real)
        # self.loss_D_person_real = self.criterionGAN(self.person_real, True)
        self.loss_D_person_real = self.criterionGAN_person(self.person_real, True)

        #Combine loss
        self.loss_D_person = (self.loss_D_person_fake + self.loss_D_person_real) * 0.5
        self.loss_D_person.backward()


    def backward_G(self):
        # First, G(A) should fake the discriminator1 and discriminator1
        # discriminator1
        fake_AB = torch.cat((self.real_A, self.fake_B), 1)
        pred_fake_image = self.netD_image.forward(fake_AB)
        # self.loss_G_GAN_image = self.criterionGAN(pred_fake_image, True)
        self.loss_G_GAN_image = self.criterionGAN_image(pred_fake_image, True)
        # Second, G(A) = B
        self.loss_G_L1 = self.criterionL1(self.fake_B, self.real_B) * self.lambda_A
        #self.loss_G_L1 = self.criterionL1(self.fake_B, self.real_B)

        pred_fake_person = self.netD_person.forward(self.person_crop_fake)
        # self.loss_G_GAN_person = self.criterionGAN(pred_fake_person, True)
        self.loss_G_GAN_person = self.criterionGAN_person(pred_fake_person, True)

        #self.loss_G_L1_person = self.criterionL1(self.person_crop_fake, self.person_crop_real)


        self.loss_G = self.loss_G_GAN_image + self.loss_G_L1 + self.loss_G_GAN_person

        self.loss_G.backward()




    def optimize_parameters(self, only_d):

        self.forward()
        self.optimizer_D_image.zero_grad()
        self.backward_D_image()
        self.optimizer_D_image.step()

        self.forward()
        self.optimizer_D_person.zero_grad()
        self.backward_D_person()
        self.optimizer_D_person.step()

        if only_d == False:
            self.forward()
            self.optimizer_G.zero_grad()
            self.backward_G()
            self.optimizer_G.step()

    def get_current_errors(self):
        return OrderedDict([('G_GAN_image', self.loss_G_GAN_image.data),
                            ('G_GAN_person', self.loss_G_GAN_person.data),
                            ('G_L1', self.loss_G_L1.data),
                            #('G_L1_person', self.loss_G_L1_person.data[0]),
                            ('D_image_real', self.loss_D_image_real.data),
                            ('D_image_fake', self.loss_D_image_fake.data),
                            ('D_person_real', self.loss_D_person_real.data),
                            ('D_person_fake', self.loss_D_person_fake.data)
                            ])

    def get_current_visuals(self):
        real_A = util.tensor2im(self.real_A.data)
        fake_B = util.tensor2im(self.fake_B.data)
        real_B = util.tensor2im(self.real_B.data)
        D2_fake = util.tensor2im(self.person_crop_fake.data)
        D2_real = util.tensor2im(self.person_crop_real.data)
        y,x,w,h = self.bbox
        display = deepcopy(real_A)
        #print(display.shape)
        display[y[0]:h[0],x[0]:w[0],:] = D2_fake
        return OrderedDict([('real_A', real_A), ('fake_B', fake_B), ('real_B', real_B), ('display', display), ('D2_fake',D2_fake),('D2_real',D2_real)])
        #return OrderedDict([('real_A', real_A), ('fake_B', fake_B), ('real_B', real_B)])

    def save(self, label):
        self.save_network(self.netG, 'G', label, self.gpu_ids)
        self.save_network(self.netD_image, 'D_image', label, self.gpu_ids)
        self.save_network(self.netD_person, 'D_person', label, self.gpu_ids)

    def update_learning_rate(self):
        lrd = self.lr / self.niter_decay
        lr = self.old_lr - lrd
        for param_group in self.optimizer_D_image.param_groups:
            param_group['lr'] = lr
        for param_group in self.optimizer_D_person.param_groups:
            param_group['lr'] = lr
        for param_group in self.optimizer_G.param_groups:
            param_group['lr'] = lr
        print('update learning rate: %f -> %f' % (self.old_lr, lr))
        self.old_lr = lr

#Downloading PreTrained Model

In [35]:
if os.path.exists(checkpoints_dir) is False:
  print("Downloading PreTrained Model")
  !mkdir checkpoints
  !gdown https://drive.google.com/u/0/uc?id=1-fncH9lKoM-YKwD8DEomPQjUv89LsX8r&export=download
  !unzip Trained_Model.zip  -d checkpoints
else:
  print('Model Already Present')

Downloading...
From (uriginal): https://drive.google.com/u/0/uc?id=1-fncH9lKoM-YKwD8DEomPQjUv89LsX8r
From (redirected): https://drive.google.com/uc?id=1-fncH9lKoM-YKwD8DEomPQjUv89LsX8r&confirm=t&uuid=217a172c-8a2d-40b4-9893-58932e0c7cf2
To: /content/gan/Trained_Model.zip
100% 445M/445M [00:06<00:00, 68.5MB/s]
Archive:  Trained_Model.zip
  inflating: checkpoints/Trained_Model/loss_log.txt  
  inflating: checkpoints/Trained_Model/2800_net_D_image.pth  
  inflating: checkpoints/Trained_Model/2800_net_D_person.pth  
  inflating: checkpoints/Trained_Model/latest_net_D_image.pth  
  inflating: checkpoints/Trained_Model/latest_net_D_person.pth  
  inflating: checkpoints/Trained_Model/latest_net_G.pth  
  inflating: checkpoints/Trained_Model/2800_net_G.pth  


#Training Script

In [ ]:


# opt = TrainOptions().parse()
data_loader = CreateDataLoader()
dataset = data_loader.load_data()
dataset_size = len(data_loader)
print('#training images = %d' % dataset_size)
#print(opt)
model = create_model()
visualizer = Visualizer()

CRITIC_ITERS = 5
total_steps = 0
iter_d = 0
only_d = False

for epoch in range(1, niter + niter_decay + 1):
    epoch_start_time = time.time()
    for i, data in enumerate(dataset):
        iter_start_time = time.time()
        total_steps += batchSize
        epoch_iter = total_steps - dataset_size * (epoch - 1)
        # y,x,w,h = data['bbox']
        # if w[0]-x[0] < 39:
        #     continue
        model.set_input(data)
#         if iter_d <= CRITIC_ITERS-1:
#             only_d = False
#         else:
#             only_d = False
        model.optimize_parameters(only_d)

        if total_steps % display_freq == 0:
            visualizer.display_current_results(model.get_current_visuals(), epoch)

        if total_steps % print_freq == 0:
            errors = model.get_current_errors()
            t = (time.time() - iter_start_time) / batchSize
            visualizer.print_current_errors(epoch, epoch_iter, errors, t)
            if display_id > 0:
              pass
                # visualizer.plot_current_errors(epoch, float(epoch_iter)/dataset_size, opt, errors)

        if total_steps % save_latest_freq == 0:
            print('saving the latest model (epoch %d, total_steps %d)' %
                  (epoch, total_steps))
            model.save('latest')
        iter_d += 1
        if iter_d == 6:
            iter_d = 0

    if epoch % save_epoch_freq == 0:
        print('saving the model at the end of epoch %d, iters %d' %
              (epoch, total_steps))
        model.save('latest')
        model.save(epoch)

    print('End of epoch %d / %d \t Time Taken: %d sec' %
          (epoch, niter + niter_decay, time.time() - epoch_start_time))

    if epoch > niter:
        model.update_learning_rate()


CustomDatasetDataLoader
dataset [AlignedDataset] was created
#training images = 6
pix2pix
False
True
---------- Networks initialized -------------
UnetGenerator(
  (model): UnetSkipConnectionBlock(
    (model): Sequential(
      (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): UnetSkipConnectionBlock(
        (model): Sequential(
          (0): LeakyReLU(negative_slope=0.2, inplace=True)
          (1): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
          (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (3): UnetSkipConnectionBlock(
            (model): Sequential(
              (0): LeakyReLU(negative_slope=0.2, inplace=True)
              (1): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
              (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (3): UnetSkipConnectionBlock(
                (model): Sequent

#Test Script
###changing some variables
###Loads the latest saved model for inference

In [ ]:
isTrain=False
nThreads = 1   # test code only supports nThreads = 1
batchSize = 1  # test code only supports batchSize = 1
serial_batches = True  # no shuffle
no_flip = True  # no flip
phase='test'
which_epoch='2800'

In [ ]:


data_loader = CreateDataLoader()
dataset = data_loader.load_data()
model = create_model()
visualizer = Visualizer()
# create website
web_dir = os.path.join(results_dir, name, '%s_%s' % (phase, which_epoch))
webpage = html.HTML(web_dir, 'Experiment = %s, Phase = %s, Epoch = %s' % (name, phase, which_epoch))
# test
for i, data in enumerate(dataset):
#    if i >= opt.how_many:
#        break
    model.set_input(data)
    model.test()
    visuals = model.get_current_visuals()
    img_path = model.get_image_paths()
    print('process image... %s' % img_path)
    visualizer.save_images(webpage, visuals, img_path)

webpage.save()